 # Assignment 6
    Group member: Akhila Ravi, Ruijia Xiong, Gregory Howard

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

import sklearn 

from sklearn import datasets
from sklearn.metrics import accuracy_score, confusion_matrix
import random
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

# Libraries related to outlier detection
from sklearn.neighbors import LocalOutlierFactor

import warnings
warnings.filterwarnings('ignore') 
sns.set(rc={'figure.figsize':(11,8)})

pd.options.display.float_format = '{:.2f}'.format

## 1. Read data from CSV

In [2]:
# Read data from csv
df19 = pd.read_csv('brfss19.csv', usecols = range(1, 250))
df21 = pd.read_csv('brfss21.csv', usecols = range(1, 250))

# Shape of the dataframes
print(df19.shape)
print(df21.shape)

(418268, 249)
(438693, 249)


In [3]:
df19.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_VEGESU1,_FRTLT1A,_VEGLT1A,_FRT16A,_VEG23A,_FRUITE1,_VEGETE1,_FLSHOT7,_PNEUMO3,_AIDTST4
0,1,1,1182019,1,18,2019,1100,2019000001,2019000001,1.00,...,114.00,1,1,1,1,0,0,2.00,1.00,2.00
1,1,1,1132019,1,13,2019,1100,2019000002,2019000002,1.00,...,121.00,1,1,1,1,0,0,1.00,1.00,2.00
2,1,1,1182019,1,18,2019,1100,2019000003,2019000003,1.00,...,164.00,1,1,1,1,0,0,1.00,2.00,2.00
3,1,1,1182019,1,18,2019,1200,2019000004,2019000004,1.00,...,NaN,9,9,1,1,1,1,9.00,9.00,NaN
4,1,1,1042019,1,4,2019,1100,2019000005,2019000005,1.00,...,178.00,1,1,1,1,0,0,2.00,1.00,2.00


In [4]:
df21.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_FRTRES1,_VEGRES1,_FRUTSU1,_VEGESU1,_FRTLT1A,_VEGLT1A,_FRT16A,_VEG23A,_FRUITE1,_VEGETE1
0,1.00,1.00,b'01192021',b'01',b'19',b'2021',1100.00,b'2021000001',2021000001.00,1.00,...,1.00,1.00,100.00,214.00,1.00,1.00,1.00,1.00,0.00,0.00
1,1.00,1.00,b'01212021',b'01',b'21',b'2021',1100.00,b'2021000002',2021000002.00,1.00,...,1.00,1.00,100.00,128.00,1.00,1.00,1.00,1.00,0.00,0.00
2,1.00,1.00,b'01212021',b'01',b'21',b'2021',1100.00,b'2021000003',2021000003.00,1.00,...,1.00,1.00,100.00,71.00,1.00,2.00,1.00,1.00,0.00,0.00
3,1.00,1.00,b'01172021',b'01',b'17',b'2021',1100.00,b'2021000004',2021000004.00,1.00,...,1.00,1.00,114.00,165.00,1.00,1.00,1.00,1.00,0.00,0.00
4,1.00,1.00,b'01152021',b'01',b'15',b'2021',1100.00,b'2021000005',2021000005.00,1.00,...,1.00,1.00,100.00,258.00,1.00,1.00,1.00,1.00,0.00,0.00


In [5]:
# Compare the different column names in the two data frame
(set(df19.columns) | set(df21.columns)) - (set(df19.columns) & set(df21.columns))

{'CHCCOPD2', 'CHCCOPD3'}

    • The different of the column names existed in the two data frame are 'CHCCOPD2', 'CHCCOPD3'

    Since the two names stand for the same thing, we just need to replace the column name df['CHCCOPD3'] in df21 to df['CHCCOPD2]

In [6]:
# Renaming one of the column to match the other
df21.rename(columns = {'CHCCOPD3':'COPD'}, inplace = True)
df19.rename(columns = {'CHCCOPD2':'COPD'}, inplace = True)

In [7]:
# Renaming INCOME3 of df21 to INCOME2 to match with df19
df21.rename(columns = {'INCOME3' : 'INCOME2'}, inplace = True)

In [8]:
# Now the two df has the same column names, we can concat the two df.
df = pd.concat([df19, df21], ignore_index=True, sort=False)
df.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_VEGESU1,_FRTLT1A,_VEGLT1A,_FRT16A,_VEG23A,_FRUITE1,_VEGETE1,_FLSHOT7,_PNEUMO3,_AIDTST4
0,1.00,1.00,1182019,1,18,2019,1100.00,2019000001,2019000001.00,1.00,...,114.00,1.00,1.00,1.00,1.00,0.00,0.00,2.00,1.00,2.00
1,1.00,1.00,1132019,1,13,2019,1100.00,2019000002,2019000002.00,1.00,...,121.00,1.00,1.00,1.00,1.00,0.00,0.00,1.00,1.00,2.00
2,1.00,1.00,1182019,1,18,2019,1100.00,2019000003,2019000003.00,1.00,...,164.00,1.00,1.00,1.00,1.00,0.00,0.00,1.00,2.00,2.00
3,1.00,1.00,1182019,1,18,2019,1200.00,2019000004,2019000004.00,1.00,...,NaN,9.00,9.00,1.00,1.00,1.00,1.00,9.00,9.00,NaN
4,1.00,1.00,1042019,1,4,2019,1100.00,2019000005,2019000005.00,1.00,...,178.00,1.00,1.00,1.00,1.00,0.00,0.00,2.00,1.00,2.00


### 1.1 Data processing with the chronic related columns
    - Depression: df['ADDEPEV3']
    - Ever Told Asthma: df['_CASTHM1']
    - Copd: df['COPD']
    - Cancer: df['CHCSCNCR'], df['CHCOCNCR']
    - Ever Told Heart Condition: df['CVDCRHD4'], df['CVDINFR4'], df['CVDSTRK3']
    - Diabetes: df['DIABETE4']

In [9]:
# For cancer, we need to combine df['CHCSCNCR'] and f['CHCOCNCR']
print(df['CHCSCNCR'].unique())
print(df['CHCOCNCR'].unique())

[ 2.  1. nan  7.  9.]
[ 2.  1.  9.  7. nan]


    • According to the Codebook Report, only 1 stands for diagnosed. So the strategy to combine the columns would be either column is then the combined column is 1, otherwise, we use 0 to stand for not diagnosed.

In [10]:
# Combine two different types of cancer into one feature
has_cncr = (df['CHCSCNCR'] == 1) | (df['CHCOCNCR'] == 1)
df['CNCR'] = has_cncr.astype(int)

df['CNCR'].value_counts()

0    708342
1    148619
Name: CNCR, dtype: int64

In [11]:
# Combine three different cardio-vascular condition into one feature
has_cvd = (df["CVDCRHD4"] == 1) | (df["CVDINFR4"] == 1) | (df["CVDSTRK3"] == 1) 
df["CVD"] = has_cvd.astype(int)
df["CVD"].value_counts() 

0    761194
1     95767
Name: CVD, dtype: int64

In [12]:
# print the percentage of unique values for cancer and heart condition
print(df['CNCR'].value_counts() / len(df['CNCR']))
print(df['CVD'].value_counts() / len(df['CVD']))

0   0.83
1   0.17
Name: CNCR, dtype: float64
0   0.89
1   0.11
Name: CVD, dtype: float64


• Before creating the two additional categorical columns, we need to clean the Nan values in the chronic condition columns

#### Missing value analysis in the chronic conditions

In [13]:
chronic_df = df[['ADDEPEV3', '_CASTHM1', 'COPD', 'CHCSCNCR', 'CHCOCNCR', 'CNCR', 'CVDCRHD4', 'CVDINFR4', 'CVDSTRK3', 'CVD', 'DIABETE4']]
print(chronic_df.shape)

(856961, 11)


In [14]:
# Missing value analysis in the chronic_df
# list all the columns and number of missing values in that column
missing_cols = chronic_df.isna().sum().reset_index()
missing_cols

,index,0
0,ADDEPEV3,13
1,_CASTHM1,0
2,COPD,11
3,CHCSCNCR,10
4,CHCOCNCR,12
5,CNCR,0
6,CVDCRHD4,10
7,CVDINFR4,12
8,CVDSTRK3,13
9,CVD,0


• We can see that there are only few missing values(<14) in these columns. We should just drop these rows.

In [16]:
df.dropna(subset=['ADDEPEV3', '_CASTHM1', 'COPD', 'CHCSCNCR', 'CHCOCNCR', 'CVDCRHD4', 'CVDINFR4', 'CVDSTRK3', 'DIABETE4'], inplace=True)

## Imputing missing values

In [17]:
chronic_conditions = ['ADDEPEV3', '_CASTHM1', 'COPD', 'CNCR', 'CVD', 
    'DIABETE4']

In [18]:
missing_values_cols = df.isna().sum()
missing_values_cols = missing_values_cols[missing_values_cols!=0].sort_values()

missing_values_cols.reset_index()

,index,0
0,CHCKDNY2,2
1,_RACEGR3,3
2,_RACEG21,3
3,_RACE,3
4,CHECKUP1,6
...,...,...
183,COLGSEX,856873
184,COLGHOUS,856873
185,WORKCFS,856941
186,HAVECFS,856941


In [19]:
threshold = 0.2 * len(df)
threshold

171388.2

### Dropping features having missing values 

In [20]:
drop_features = missing_values_cols[missing_values_cols > threshold].index
drop_features

Index(['_CHISPNC', 'SAFETIME', 'PVTRESD3', 'CELLFON5', 'CTELNUM1', 'CADULT1',
       'CSTATE1', 'HHADULT', 'CELLSEX', 'LANDLINE',
       ...
       'CSRVCTL2', 'HAVEHEPC', 'TRETHEPC', 'PRIRHEPC', 'MEDSHEPB', 'COLGSEX',
       'COLGHOUS', 'WORKCFS', 'HAVECFS', 'TOLDCFS'],
      dtype='object', length=132)

In [21]:
len(drop_features)

132

In [22]:
# Dropping the features having more than 20% of missing values
df = df.drop(drop_features, axis = 1)

### Imputing values for features

In [23]:
impute_features = missing_values_cols[missing_values_cols < threshold].index
impute_features

Index(['CHCKDNY2', '_RACEGR3', '_RACEG21', '_RACE', 'CHECKUP1', '_PRACE1',
       '_MRACE1', 'MENTHLTH', 'EDUCA', 'RENTHOM1', 'GENHLTH', 'PHYSHLTH',
       'MARITAL', 'CPDEMO1B', 'VETERAN3', 'QSTLANG', 'EMPLOY1', '_MICHD',
       'CHILDREN', '_METSTAT', '_URBSTAT', 'WEIGHT2', 'EXERANY2', 'HEIGHT3',
       'DEAF', 'BLIND', 'DECIDE', 'DIFFWALK', 'DIFFDRES', 'DIFFALON',
       'SMOKE100', 'USENOW3', 'ALCDAY5', 'HTM4', 'HTIN4', 'FLUSHOT7', 'FRUIT2',
       'PNEUVAC4', 'FRUITJU2', 'FVGREEN1', 'WTKG3', 'HIVTST7', '_AIDTST4',
       'FRENCHF1', 'POTATOE1', 'VEGETAB2', 'FTJUDA2_', 'FRUTDA2_', '_BMI5CAT',
       '_BMI5', 'GRENDA1_', 'FRNCHDA_', 'POTADA1_', 'VEGEDA2_', '_FRUTSU1',
       '_VEGESU1'],
      dtype='object')

#### Using mean imputer for numerical values

In [24]:
from sklearn.impute import SimpleImputer

mean_imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')

# Impute an empty value by replacing it with mean 
def impute_by_mean(col_name):
    df[col_name] = mean_imputer.fit_transform(df[col_name].values.reshape(-1, 1))


In [25]:
# Imputing missing values for features having < 20% missing values
for col in impute_features:
    impute_by_mean(col)

## Identifying feature and response space

### Response space

#### 1. Creating a binary categorical response column

In [27]:
### 1.2 Create two additional categorical columns - CMB_bianry, CMN_multi
has_chronic = (df['CNCR'] == 1)  | (df['ADDEPEV3'] == 1) | (df['_CASTHM1'] == 1) | (df['COPD'] == 1) | (df['CVD'] == 1) | (df['DIABETE4'] == 1) | (df['DIABETE4'] == 2)
df['CMB_binary'] = has_chronic.astype(int)
df['CMB_binary'].unique()

array([1, 0])

In [28]:
df["CMB_binary"].value_counts()

1    827041
0     29900
Name: CMB_binary, dtype: int64

#### 2. Creating a multiclass response column

In [29]:
# Creating a subset of chronic conditions
cf = df[chronic_conditions]

In [30]:
# Replace values of 1 and 2 with 1 and rest with 0
cf["DIABETE4"] = cf["DIABETE4"].apply(lambda x: 1 if (x == 1) | (x == 2) else 0)

# Replace values of 1 with 1 and rest with 0 in other features
for col in chronic_conditions:
    cf[col] = cf[col].apply(lambda x: 1 if x == 1 else 0)

In [31]:
# Determine the number of chronic conditions an individual has
num_conditions = cf.sum(axis=1)

# Creating a new response column with the num of conditions
df['CMB_multiclass'] = pd.cut(num_conditions, bins=[0, 1, 2, 3, float('inf')], labels=[0, 1, 2, '3+'])


In [32]:
df["CMB_multiclass"].value_counts()

0     450760
1     250601
2      91764
3+     33916
Name: CMB_multiclass, dtype: int64

In [33]:
response_cols = ["CMB_binary", "CMB_multiclass"]

## Analysis Part 1: